# Sectors Review

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [2]:
data = pd.read_csv('../../data/sp500_Company_Overview.csv', index_col=0)
data.head()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740.0,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,"3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US",...,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08,2024-06-12,2024-05-23
AOS,Common Stock,Smith AO Corporation,A. O. Smith Corporation is an American manufac...,91142.0,NYSE,USD,USA,MANUFACTURING,HOUSEHOLD APPLIANCES,"11270 WEST PARK PLACE, MILWAUKEE, WI, US",...,3.248,15.19,1.141,92.44,63.34,83.92,81.37,1.207840e+08,2024-08-15,2024-07-31
ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"100 ABBOTT PARK ROAD, ABBOTT PARK, IL, US",...,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09,2024-08-15,2024-07-15
ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"1 NORTH WAUKEGAN ROAD, NORTH CHICAGO, IL, US",...,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09,2024-08-15,2024-07-15
ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373.0,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1 GRAND CANAL SQUARE, GRAND CANAL HARBOUR, DUB...",...,3.041,18.21,1.227,384.33,277.47,301.44,329.36,6.263840e+08,2024-08-15,2024-07-11


In [12]:
data.drop(['AssetType',
           'CIK',
           'Description',
           'Currency',
           'Country',
           'Address',
           'DividendPerShare'
          ], inplace=True, axis=1)

In [18]:
data.replace('None', np.nan, inplace=True)

In [19]:
data['Sector'].unique()

array(['LIFE SCIENCES', 'MANUFACTURING', 'TRADE & SERVICES', 'TECHNOLOGY',
       'ENERGY & TRANSPORTATION', 'FINANCE', 'REAL ESTATE & CONSTRUCTION'],
      dtype=object)

In [20]:
data.groupby(['Sector']).count()['Name'].to_frame().sort_values('Name', ascending=False)

,Name
Sector,
MANUFACTURING,96
LIFE SCIENCES,80
ENERGY & TRANSPORTATION,76
TRADE & SERVICES,75
TECHNOLOGY,69
FINANCE,65
REAL ESTATE & CONSTRUCTION,42


## LIFE SCIENCES

In [21]:
lf = data[data['Sector']=='LIFE SCIENCES'].copy()
lf.drop('Sector', axis=1, inplace=True)

In [22]:
lf.head()

,Name,Exchange,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,3M Company,NYSE,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,December,2024-03-31,5.808631e+10,8146000000.0,NaN,1.905,8.79,...,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08,2024-06-12,2024-05-23
ABT,Abbott Laboratories,NYSE,PHARMACEUTICAL PREPARATIONS,December,2024-03-31,1.804344e+11,10485000000.0,32.72,5.99,22.31,...,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09,2024-08-15,2024-07-15
ABBV,AbbVie Inc,NYSE,PHARMACEUTICAL PREPARATIONS,December,2024-03-31,3.065727e+11,26116000000.0,51.52,0.449,4.535,...,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09,2024-08-15,2024-07-15
A,Agilent Technologies Inc,NYSE,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,October,2024-04-30,3.892675e+10,1612000000.0,31.62,2.701,21.31,...,6.24,24.88,1.049,155.06,96.30,136.10,133.04,2.917610e+08,2024-07-24,2024-07-02
APD,Air Products and Chemicals Inc,NYSE,INDUSTRIAL INORGANIC CHEMICALS,September,2024-03-31,5.525416e+10,4115100000.0,22.45,1.372,67.08,...,4.989,13.79,0.822,299.60,209.25,263.88,257.75,2.223060e+08,2024-11-12,2024-10-01
